# CICIDS

### **PCA.ipynb**

The CICIDS2017 dataset is a comprehensive dataset for network intrusion detection, created by the Canadian Institute for Cybersecurity. It includes a diverse set of attack scenarios and normal traffic, making it suitable for training and evaluating intrusion detection systems.

The dataset includes various types of attacks such as Brute Force, Heartbleed, Botnet, DoS (Denial of Service), DDoS (Distributed Denial of Service), Web attacks, and Infiltration of the network from inside.

In this notebook we attempt to replicate the usage of PCA explained in the paper ["Towards Model Generalization for Intrusion Detection: Unsupervised Machine Learning Techniques"](https://biblio.ugent.be/publication/8725248)

In [1]:
model_name = "PCA"

In [2]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## Step 1. Read data and import necessary libraries

In [3]:
import pandas as pd
df_train = pd.read_csv("./data/concatenated/concat.csv")

In [4]:
df_train.head(5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,5.000000e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,6.666667e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [5]:
# df_train.shape

In [6]:
# df_train.info()

In [7]:
# df_train.describe()

In [8]:
# Remove leading and trailing whitespaces from column names
df_train.columns = df_train.columns.str.strip()

In [9]:
df_train.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

## Step 2. Data Cleaning

### A. Missing values

In [10]:
print(df_train.isna().sum().sum())

1358


In [11]:
df_train.dropna(subset=["Flow Bytes/s"], inplace=True)

In [12]:
print(df_train.isna().sum().sum())

0


### Inf. values

In [13]:
import numpy as np
df_train = df_train.replace([np.inf, -np.inf], np.nan).dropna()

## Step 3. Data Preparation

### A. Normalise numeric features

In [14]:
# # Get all numerical columns
# numerical_columns = df_train.select_dtypes(include="number").columns

In [15]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df_train[numerical_columns] = scaler.fit_transform(df_train[numerical_columns])

### B. Map Labels to Multi-class

In [16]:
df_train["Label"].value_counts()

Label
BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [17]:
# BINARY MAPPING - (Benign/Attack)
attack_mapping = {
	"BENIGN": 0,
	"DoS Hulk": 1,
	"PortScan": 1,
	"DDoS": 1,
	"DoS GoldenEye": 1,
	"FTP-Patator": 1,
	"SSH-Patator": 1,
	"DoS slowloris": 1,
	"DoS Slowhttptest": 1,
	"Bot": 1,
	"Web Attack � Brute Force": 1,
	"Web Attack � XSS": 1,
	"Infiltration": 1,
	"Web Attack � Sql Injection": 1,
	"Heartbleed": 1,
}

df_train["Label"] = df_train["Label"].map(attack_mapping)

In [18]:
df_train["Label"].value_counts()

Label
0    2271320
1     556556
Name: count, dtype: int64

### C. Data Splitting

In [19]:
X = df_train.drop(columns="Label")
y = df_train["Label"]

In [20]:
# Standardize the data (PCA assumes zero-mean, unit variance)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Step 4. Model

#### A. Defining the model

In [21]:
from sklearn.decomposition import PCA

# Train PCA on **only benign data**
X_benign = X_scaled[y == 0]

# Choose the number of principle components
n_components = 10 # This is a hyperparameter that should be optimised

### B. Training the model

In [22]:
# Fit PCA on benign traffic
pca = PCA(n_components=n_components)
pca.fit(X_benign)

# Transform and then inverse transform the entire dataset
X_pca = pca.transform(X_scaled)
X_reconstructed = pca.inverse_transform(X_pca)

### C. Reconstruction error

In [23]:
# Compute Reconstruction Error (Sum of Squared Errors - SSE)
reconstruction_error = np.sum((X_scaled - X_reconstructed) ** 2, axis=1)

# Determine anomaly threshold (e.g., 95th percentile of benign errors)
threshold = np.percentile(reconstruction_error[y == 0], 95)

### D. Make predictions

In [24]:
# Predict anomalies
y_pred = (reconstruction_error > threshold).astype(int)

### E. Evaluating the model

In [ ]:
# Evaluate the model
from sklearn.metrics import roc_auc_score, classification_report

print(f"AUC Score: {roc_auc_score(y, reconstruction_error):.4f}\n")
print(f"Classification Report:\n{classification_report(y, y_pred)}")

AUC Score: 0.7983452800618147
              precision    recall  f1-score   support

           0       0.88      0.95      0.91   2271320
           1       0.70      0.47      0.56    556556

    accuracy                           0.85   2827876
   macro avg       0.79      0.71      0.74   2827876
weighted avg       0.84      0.85      0.84   2827876

